In [1]:
import os
os.environ["TRITON_INTERPRET"] = "1"

In [78]:
# Repro ready
import sys
sys.path.append('../')
import torch
#from model_triton import t_scaled_dot_prod_attn_fwd3 #, t_scaled_dot_prod_attn_fwd3_t

#BS, H, N, D = 2, 2, 512, 64
#BS, H, N, D = 1, 1, 16, 16
#BS, H, N, D = 1, 1, 512, 64
#BS, H, N, D = 1, 1, 512, 16 #bug
#BS, H, N, D = 1, 1, 128, 16 #not bug
#BS, H, N, D = 1, 1, 256, 16 #bug
BS, H, N, D = 1, 1, 32, 16 #bug
# BS, H, N, D = 1, 1, 16, 16
# qkv = torch.randn((BS, H, 3, N, D), device="cuda")
qkv = torch.randn((1, H, 3, N, D), device="cuda").expand(BS, H, 3, N, D)
mask = torch.tril(torch.ones((N,N), dtype=torch.bool, device="cuda")).unsqueeze(0).repeat(BS, 1, 1)
train=False #True
p_gen_aux = 42
N_RUNS = 1 #10

# qkv[:,:,2, :, :] = 1 # set V to 1s, so I can interepet output easily
# qkv[:,:,:, :, :] = 1 # set V to 1s, so I can interepet output easily

# full mask
mask[0, 0:, :]=0 # This mask is not supported 

import math
from model_triton import t_log_softmax_fwd, t_dropout_fwd
def t_softmax_attn_fwd(q, k, mask, train, p_gen_aux=None):
    D = q.shape[-1]
    attn = torch.matmul(q, torch.transpose(k, -2, -1))
    attn = attn / math.sqrt(D)
    attn = torch.where(torch.unsqueeze(mask,dim=1), attn, torch.full_like(attn, -1e9)) # Note, instead of usign -jnp.inf, which results in NaNs (NIT: probably better to use jax.numpy.finfo)
    #print(attn)
    sa = torch.exp(t_log_softmax_fwd(attn))
    #print(sa)
    sa = t_dropout_fwd(sa, train, p_gen_aux)
    #print(sa)
    return sa

def t_scaled_dot_prod_attn_fwd3(qkv, mask, train=True, p_gen_aux=None): # inputs: BS x H x 3 x N x D, mask: BS x N(q) x N(k)
    q, k, v = torch.unbind(qkv, dim=2) # BS x H x N x D
    softmaxed_attn = t_softmax_attn_fwd(q, k, mask, train, p_gen_aux)
    return torch.matmul(softmaxed_attn, v), [softmaxed_attn] # output: BS x H x N x D

result, _ = t_scaled_dot_prod_attn_fwd3(qkv, mask, train, p_gen_aux)
# print(result)

# result2, _ = t_scaled_dot_prod_attn_fwd3_t(qkv, mask, train, p_gen_aux)

# assert torch.allclose(result, result2, rtol=5e-2, atol=1e-3), (result.shape, result2.shape, result[:2, :2, -4:, -4:], result2[:2, :2, -4:, -4:])

In [83]:
import triton
import triton.language as tl

from model_triton import gelu_k, T_DROPOUT_RATE, dropout_k

# This will work for moderate size of D: it's tiling along N dimension of Q, and along N dimension of K_T&V.
# It doesn't tile along D dimension.
# Different program per BS_H item (reshape of BS and H in one dim, and one program per this dim)
@triton.jit
def t_scaled_dot_prod_attn_fwd3_k(q_ptr, k_t_ptr, v_ptr, mask_ptr, output_ptr, acts0_ptr, acts1_ptr,
                q_stride0, q_stride1, q_stride2, k_t_stride0, k_t_stride1, k_t_stride2,
                v_stride0, v_stride1, v_stride2, mask_stride0, mask_stride1,
                output_stride0, output_stride1, output_stride2, acts0_stride0, acts1_stride0,
                train, p_gen_aux,
                BS_H, N:tl.constexpr, D,
                BLOCK_SIZE_Q_N: tl.constexpr, BLOCK_SIZE_K_T_N: tl.constexpr, BLOCK_SIZE_D: tl.constexpr,
                Q_N_BLCKS: tl.constexpr,
                #num_stages: tl.constexpr
                ):
    # Matching PyTorch's fp32 dtype ( see https://github.com/triton-lang/triton/issues/4574)
    ASM: tl.constexpr = "cvt.rna.tf32.f32 $0, $1;"
    
    sqrt_D = tl.sqrt(D.to(tl.float32)) # TODO T: extract from this method?
    bs_h_start = tl.program_id(0)
    bs_h_step = tl.num_programs(0)
    for bs_h_pid in tl.range(bs_h_start, BS_H, bs_h_step): #, num_stages):
        bs_h_q_ptr = q_ptr + bs_h_pid * q_stride0
        bs_h_k_t_ptr = k_t_ptr + bs_h_pid * k_t_stride0    
        bs_h_v_ptr = v_ptr + bs_h_pid * v_stride0        
        bs_h_output_ptr = output_ptr + bs_h_pid * output_stride0
        bs_h_acts0_ptr = acts0_ptr + bs_h_pid * acts0_stride0
        bs_h_acts1_ptr = acts1_ptr + bs_h_pid * acts1_stride0     
        
        for q_n_step in tl.static_range(0, Q_N_BLCKS):          
            q_n_offsets = q_n_step * BLOCK_SIZE_Q_N + tl.arange(0, BLOCK_SIZE_Q_N)            
            d_offsets = tl.arange(0, BLOCK_SIZE_D)
            # TODO T: Do I need modulo n, modulo m operations? 
            q_n_offsets_mod = q_n_offsets % N
            d_offsets_mod = d_offsets %D
            
            # Load Q blck once
            q_blck_ptr = bs_h_q_ptr + q_n_offsets_mod[:,None] * q_stride1 + d_offsets_mod[None, :] * q_stride2
            q_blck_mask = (q_n_offsets[:,None] < N) & (d_offsets[None, :] < D)
            q_blck = tl.load(q_blck_ptr, mask=q_blck_mask, other=0.0) 
            
            # Use FlashAttention naming: ms - logits' max, ls - logit's sumexp
            ms = tl.full((BLOCK_SIZE_Q_N, 1), -1e9, tl.float32)
            ls = tl.zeros_like(ms)
            output = tl.zeros((BLOCK_SIZE_Q_N, BLOCK_SIZE_D), dtype=tl.float32)
            
            # ASSUMES CAUSAL MASK FOR NOW
            # This is somehow limited suppport for now. I only tested this for BLOCK_SIZE_Q_N = 2x BLOCK_SIZE_K_T_N
            #k_t_n_step_end = min(tl.cdiv(N, BLOCK_SIZE_K_T_N), (q_n_step+1) *tl.cdiv(BLOCK_SIZE_Q_N, BLOCK_SIZE_K_T_N))
            k_t_n_step_end = tl.cdiv(N, BLOCK_SIZE_K_T_N)
            
            for k_t_n_step in range(0, k_t_n_step_end):
                k_t_n_offsets = k_t_n_step * BLOCK_SIZE_K_T_N + tl.arange(0, BLOCK_SIZE_K_T_N) 
                k_t_n_offsets_mod = k_t_n_offsets % N
                
                # Q * K^T
                k_blck_ptr = bs_h_k_t_ptr + d_offsets_mod[:,None] * k_t_stride1 + k_t_n_offsets_mod[None, :] * k_t_stride2
                k_blck_mask = (d_offsets[:, None] < D) & (k_t_n_offsets[None, :] < N)
                k_blck = tl.load(k_blck_ptr, mask=k_blck_mask, other=0.0)
                # Matching PyTorch's fp32 dtype ( see https://github.com/triton-lang/triton/issues/4574)
                #q_blck = tl.inline_asm_elementwise(ASM, "=r, r", [q_blck], dtype=tl.float32, is_pure=True, pack=1)
                #k_blck = tl.inline_asm_elementwise(ASM, "=r, r", [k_blck], dtype=tl.float32, is_pure=True, pack=1)
                acc = tl.dot(q_blck, k_blck)

                # /sqrt(D) + Mask + Softmax + Dropout (with keeping updating ms&ls following FlashAttention's paper)
                acc = acc / sqrt_D
                mask_blck_ptr = mask_ptr + q_n_offsets_mod[:,None] * mask_stride0 + k_t_n_offsets_mod[None, :] * mask_stride1
                mask_mask = (q_n_offsets[:,None] <N) & (k_t_n_offsets[None, :]<N)
                mask_blck = tl.load(mask_blck_ptr, mask=mask_mask, other= 0.0)
#                 print("k_t_n_step, mask_blck", k_t_n_step, mask_blck)
                acc = tl.where(mask_blck, acc, -1e9)
                blck_ms = tl.max(acc, axis=1, keep_dims=True)
#                 print("blck_ms", blck_ms)
                n_ms = tl.maximum(ms, blck_ms)
#                 print("n_ms", n_ms)
                acc = tl.exp(acc - blck_ms)
#                 print("acc", acc)
                blck_ls = tl.sum(acc, axis=1, keep_dims=True)   
#                 print("blck_ls", blck_ls)
                n_ls = tl.exp(ms - n_ms) * ls + tl.exp(blck_ms - n_ms) * blck_ls
#                 print("n_ls", n_ls)
                #print(acc)
                # TODO T: confirm that this is different enough seed per row (assumes that D_PID always equals to 0)
                acc = dropout_k(acc, train, p_gen_aux+bs_h_pid, q_n_offsets[:,None] + k_t_n_offsets[None, :])
#                 print("acc", acc)
                #print(acc)
                
                # * V
                v_blck_ptr = bs_h_v_ptr + k_t_n_offsets_mod[:,None] * v_stride1 + d_offsets_mod[None, :] * v_stride2
                v_blck_mask = (k_t_n_offsets[:, None] < N) & (d_offsets[None, :]<D)
                v_blck = tl.load(v_blck_ptr, mask=v_blck_mask, other=0.0)
                #print("\n")
                #print("q_n_step, k_t_n_step, v_blck", q_n_step, k_t_n_step, v_blck)
                #v_blck = tl.inline_asm_elementwise(ASM, "=r, r", [v_blck], dtype=tl.float32, is_pure=True, pack=1)
                
                output = 1/n_ls * (ls * tl.exp(ms - n_ms) * output + tl.exp(blck_ms - n_ms) * tl.dot(acc, v_blck))
#                 print("output", output)
                ms, ls = n_ms, n_ls
                
            tl.store(bs_h_acts0_ptr + q_n_offsets, ms.reshape(BLOCK_SIZE_Q_N), mask=q_n_offsets<N)
            tl.store(bs_h_acts1_ptr + q_n_offsets, ls.reshape(BLOCK_SIZE_Q_N), mask=q_n_offsets<N)  
            output_blck_ptr = bs_h_output_ptr + q_n_offsets[:,None] * output_stride1 + d_offsets[None, :] * output_stride2
            output_mask = (q_n_offsets[:,None] <N) & (d_offsets[None, :]<D)
            tl.store(output_blck_ptr, output, mask=output_mask)
    

def t_scaled_dot_prod_attn_fwd3_t(qkv:torch.Tensor, mask:torch.Tensor, train=True, p_gen_aux=None):
    q, k, v = torch.unbind(qkv, dim=2) # BS x H x N x D
    BS, H, N, D = q.shape
    
    q = q.reshape(BS*H, N, D)
    k = k.reshape(BS*H, N, D)
    v = v.reshape(BS*H, N, D)
    mask = mask[0] # Asumme mask being the same across rows. TODO XXX: make that assumption throughput the code
    
    output = torch.zeros_like(q)
    acts0 = torch.empty((BS*H, N), device=q.device)
    acts1 = torch.empty_like(acts0)
    
    # TODO T: check if some matrices are contiguous?
    grid = (min(BS*H, 80),)

    # Tuned params given num_warps=8, and BS, H, N, D = 8, 12, 512, 64
    num_warps = 8
    num_stages = 2 # TODO T: I don't think this helps
    BLOCK_SIZE_Q_N = 16 #128
    BLOCK_SIZE_K_T_N = 16 #64
    BLOCK_SIZE_D = triton.next_power_of_2(D)
    
    # We enforce causal masking for now, but the assert below cost too much perf
    #assert torch.allclose(mask, torch.tril(torch.ones((N, N), device=mask.device, dtype=torch.bool))), "Assumes causal mask"
    assert BLOCK_SIZE_Q_N >= BLOCK_SIZE_K_T_N, "The Q block size needs to be bigger/equal than the K block size. Due to the limited support for levarging causal mask"
    
    assert N % BLOCK_SIZE_Q_N==0, "Given the limited support, N has be dividable by the Q block size"
    assert N % BLOCK_SIZE_K_T_N==0, "Given the limited support, N has be dividable by the K block size"    

#     print(v.shape, v[0,:,:4])
    
    if not train:
        p_gen_aux = 0 # Need to mock some value for triton to compile the kernel without errors
    k_t = torch.transpose(k, -2, -1)
    t_scaled_dot_prod_attn_fwd3_k[grid](
        q, k_t, v, mask, output, acts0, acts1,
        q.stride(0), q.stride(1), q.stride(2), k_t.stride(0), k_t.stride(1), k_t.stride(2), 
        v.stride(0), v.stride(1), v.stride(2),
        mask.stride(0), mask.stride(1), output.stride(0), output.stride(1), output.stride(2),
        acts0.stride(0), acts1.stride(0),
        train, p_gen_aux,
        BS*H, N, D,
        BLOCK_SIZE_Q_N=BLOCK_SIZE_Q_N, BLOCK_SIZE_K_T_N = BLOCK_SIZE_K_T_N, BLOCK_SIZE_D=BLOCK_SIZE_D,
        Q_N_BLCKS = triton.cdiv(N, BLOCK_SIZE_Q_N), # TODO T: Is there a way to do that cdiv inside kernel?
        num_warps=num_warps, num_stages=num_stages)
    
    output = output.reshape(BS, H, N, D)
    return output, [acts0.reshape(BS, H, N), acts1.reshape(BS, H, N), output]


result2, _ = t_scaled_dot_prod_attn_fwd3_t(qkv, mask, train, p_gen_aux)

assert torch.allclose(result, result2, rtol=5e-2, atol=1e-3), (result.shape, result2.shape, result[:2, :2, -4:, -4:], result2[:2, :2, -4:, -4:])

In [76]:
result

tensor([[[[-1.3309e-01,  2.6458e-01,  2.4963e-02, -1.3783e-01,  7.1695e-02,
           -2.3369e-01, -1.9723e-04, -3.0952e-01, -3.9327e-02,  1.6426e-01,
           -1.4887e-01, -1.8342e-02,  1.4528e-01,  8.2294e-02,  3.3704e-01,
           -4.2749e-03],
          [-1.3309e-01,  2.6458e-01,  2.4963e-02, -1.3783e-01,  7.1695e-02,
           -2.3369e-01, -1.9723e-04, -3.0952e-01, -3.9327e-02,  1.6426e-01,
           -1.4887e-01, -1.8342e-02,  1.4528e-01,  8.2294e-02,  3.3704e-01,
           -4.2749e-03],
          [-1.3309e-01,  2.6458e-01,  2.4963e-02, -1.3783e-01,  7.1695e-02,
           -2.3369e-01, -1.9723e-04, -3.0952e-01, -3.9327e-02,  1.6426e-01,
           -1.4887e-01, -1.8342e-02,  1.4528e-01,  8.2294e-02,  3.3704e-01,
           -4.2749e-03],
          [-1.3309e-01,  2.6458e-01,  2.4963e-02, -1.3783e-01,  7.1695e-02,
           -2.3369e-01, -1.9723e-04, -3.0952e-01, -3.9327e-02,  1.6426e-01,
           -1.4887e-01, -1.8342e-02,  1.4528e-01,  8.2294e-02,  3.3704e-01,
           -4

In [77]:
result2

tensor([[[[-1.3309e-01,  2.6458e-01,  2.4963e-02, -1.3783e-01,  7.1695e-02,
           -2.3369e-01, -1.9722e-04, -3.0952e-01, -3.9327e-02,  1.6426e-01,
           -1.4887e-01, -1.8342e-02,  1.4528e-01,  8.2294e-02,  3.3704e-01,
           -4.2749e-03],
          [-1.3309e-01,  2.6458e-01,  2.4963e-02, -1.3783e-01,  7.1695e-02,
           -2.3369e-01, -1.9722e-04, -3.0952e-01, -3.9327e-02,  1.6426e-01,
           -1.4887e-01, -1.8342e-02,  1.4528e-01,  8.2294e-02,  3.3704e-01,
           -4.2749e-03],
          [-1.3309e-01,  2.6458e-01,  2.4963e-02, -1.3783e-01,  7.1695e-02,
           -2.3369e-01, -1.9722e-04, -3.0952e-01, -3.9327e-02,  1.6426e-01,
           -1.4887e-01, -1.8342e-02,  1.4528e-01,  8.2294e-02,  3.3704e-01,
           -4.2749e-03],
          [-1.3309e-01,  2.6458e-01,  2.4963e-02, -1.3783e-01,  7.1695e-02,
           -2.3369e-01, -1.9722e-04, -3.0952e-01, -3.9327e-02,  1.6426e-01,
           -1.4887e-01, -1.8342e-02,  1.4528e-01,  8.2294e-02,  3.3704e-01,
           -4